In [4]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np
df = pd.read_csv('data/diabetic_data.csv', encoding = 'latin1',low_memory=False)

In [89]:
df.head()

,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,num_medications,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,Caucasian,Female,[0-10),6,25,1,1,41,0,1,...,No,No,No,No,No,No,No,No,No,0
2,AfricanAmerican,Female,[20-30),1,1,7,2,11,5,13,...,No,No,No,No,No,No,No,No,Yes,0
3,Caucasian,Male,[30-40),1,1,7,2,44,1,16,...,No,Up,No,No,No,No,No,Ch,Yes,0
4,Caucasian,Male,[40-50),1,1,7,1,51,0,8,...,No,Steady,No,No,No,No,No,Ch,Yes,0
6,Caucasian,Male,[60-70),3,1,2,4,70,1,21,...,No,Steady,No,No,No,No,No,Ch,Yes,0


In [6]:
max(df.number_diagnoses.value_counts())

49474

In [126]:
from sklearn.model_selection import train_test_split

df = pd.read_csv('data/diabetic_data.csv', encoding = 'latin1',low_memory=False)
df.readmitted[df.readmitted == 'NO' ] = 0
df.readmitted[df.readmitted == '<30' ] = 1
df = df.drop(df[df.readmitted == '>30'].index)
df.drop(['encounter_id', 'patient_nbr', 'weight', 'payer_code', 'medical_specialty'], axis=1, inplace=True)
#df.dropna(axis=1, how='all')
y = df['readmitted']
X = df.drop('readmitted', axis=1)
X_train, X_test = train_test_split(X, test_size=0.2)
y_train, y_test = train_test_split(y, test_size=0.2)
df_train, df_test = train_test_split(df, test_size=0.2)
#X = pd.get_dummies(X)

In [8]:
y.value_counts()

0    54864
1    11357
Name: readmitted, dtype: int64

In [123]:
#for a in X_train.columns.values:
    #print(a)

In [91]:
X.head()

,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,num_medications,...,examide,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed
0,Caucasian,Female,[0-10),6,25,1,1,41,0,1,...,No,No,No,No,No,No,No,No,No,No
2,AfricanAmerican,Female,[20-30),1,1,7,2,11,5,13,...,No,No,No,No,No,No,No,No,No,Yes
3,Caucasian,Male,[30-40),1,1,7,2,44,1,16,...,No,No,Up,No,No,No,No,No,Ch,Yes
4,Caucasian,Male,[40-50),1,1,7,1,51,0,8,...,No,No,Steady,No,No,No,No,No,Ch,Yes
6,Caucasian,Male,[60-70),3,1,2,4,70,1,21,...,No,No,Steady,No,No,No,No,No,Ch,Yes


In [127]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler


encoders = dict()
numeric_headers = ["time_in_hospital", "num_lab_procedures", "num_procedures", "num_medications", "number_outpatient", "number_emergency", "number_inpatient", "number_diagnoses"]
categorical_headers = ['race',
 'gender',
 'age',
 'admission_type_id',
 'discharge_disposition_id',
 'admission_source_id',
 'diag_1',
 'diag_2',
 'diag_3',
 'max_glu_serum',
 'A1Cresult',
 'metformin',
 'repaglinide',
 'nateglinide',
 'chlorpropamide',
 'glimepiride',
 'acetohexamide',
 'glipizide',
 'glyburide',
 'tolbutamide',
 'pioglitazone',
 'rosiglitazone',
 'acarbose',
 'miglitol',
 'troglitazone',
 'tolazamide',
 'examide',
 'citoglipton',
 'insulin',
 'glyburide-metformin',
 'glipizide-metformin',
 'glimepiride-pioglitazone',
 'metformin-rosiglitazone',
 'metformin-pioglitazone',
 'change',
 'diabetesMed']

for col in categorical_headers:
    #X_train[col] = X_train[col].str.strip()
    #X_test[col] = X_test[col].str.strip()
    #df[col] = df[col].str.strip()

    encoders[col] = LabelEncoder()
    df_train[col+'_int'] = encoders[col].fit_transform(df_train[col])
    df_test[col+'_int'] = encoders[col].transform(df_test[col])
    #df[col+'_int'] = encoders[col].fit_transform(df[col])

for col in numeric_headers:
    df_train_[col] = df_train_[col].astype(np.float)
    df_test_[col] = df_test_[col].astype(np.float)
    #df[col] = df[col].astype(np.float)
    ss = StandardScaler()
    df_train_[col] = ss.fit_transform(df_train_[col].values.reshape(-1, 1))
    df_test_[col] = ss.transform(df_test_[col].values.reshape(-1, 1))
    #df[col] = ss.fit_transform(df[col].values.reshape(-1, 1))
    
X_train.head()

ValueError: y contains new labels: [10]

In [15]:
from sklearn import metrics as mt

In [16]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Input
from keras.layers import Embedding, Flatten, Merge, concatenate
from keras.models import Model

In [19]:
# This returns a tensor
inputs = Input(shape=(X_train_.shape[1],))

# a layer instance is callable on a tensor, and returns a tensor
x = Dense(units=10, activation='relu')(inputs)
predictions = Dense(1,activation='sigmoid')(x)

# This creates a model that includes
# the Input layer and three Dense layers
model = Model(inputs=inputs, outputs=predictions)

In [20]:
# combine the features into a single large matrix
X_train = X_train_.values
X_test = X_test_.values
y_train = y_train_.astype(np.int)
y_test = y_test_.astype(np.int)

# This returns a tensor
inputs = Input(shape=(X_train.shape[1],))

# a layer instance is callable on a tensor, and returns a tensor
x = Dense(units=10, activation='relu')(inputs)
predictions = Dense(1,activation='sigmoid')(x)

# This creates a model that includes
# the Input layer and three Dense layers
model = Model(inputs=inputs, outputs=predictions)

In [21]:
model.compile(optimizer='sgd',
              loss='mean_squared_error',
              metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 2245)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                22460     
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 11        
Total params: 22,471
Trainable params: 22,471
Non-trainable params: 0
_________________________________________________________________


In [22]:
%%time

model.fit(X_train, y_train, epochs=10, batch_size=50, verbose=0)

from sklearn import metrics as mt
yhat = np.round(model.predict(X_test))
print(mt.confusion_matrix(y_test,yhat),mt.accuracy_score(y_test,yhat))

WARNING (theano.configdefaults): install mkl with `conda install mkl-service`: No module named 'mkl'


KeyboardInterrupt: 

In [42]:
print(mt.accuracy_score(y_test,yhat))

0.832870258517


In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 11972458854901381395
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 6789848269
locality {
  bus_id: 1
}
incarnation: 14834288962779265312
physical_device_desc: "device: 0, name: GeForce GTX 1080, pci bus id: 0000:01:00.0, compute capability: 6.1"
]


In [23]:
from keras import backend as K
import os

def set_keras_backend(backend):

    if K.backend() != backend:
        os.environ['KERAS_BACKEND'] = backend
        reload(K)
        assert K.backend() == backend

set_keras_backend("tensorflow")

NameError: name 'reload' is not defined

## Starting work on the actual deep and wide network

In [102]:
categorical_headers_ints = [x+'_int' for x in categorical_headers]
df_num =  df[numeric_headers].values
print(categorical_headers_ints)

['race_int', 'gender_int', 'age_int', 'admission_type_id_int', 'discharge_disposition_id_int', 'admission_source_id_int', 'diag_1_int', 'diag_2_int', 'diag_3_int', 'max_glu_serum_int', 'A1Cresult_int', 'metformin_int', 'repaglinide_int', 'nateglinide_int', 'chlorpropamide_int', 'glimepiride_int', 'acetohexamide_int', 'glipizide_int', 'glyburide_int', 'tolbutamide_int', 'pioglitazone_int', 'rosiglitazone_int', 'acarbose_int', 'miglitol_int', 'troglitazone_int', 'tolazamide_int', 'examide_int', 'citoglipton_int', 'insulin_int', 'glyburide-metformin_int', 'glipizide-metformin_int', 'glimepiride-pioglitazone_int', 'metformin-rosiglitazone_int', 'metformin-pioglitazone_int', 'change_int', 'diabetesMed_int']


In [104]:
# we need to create separate sequential models for each embedding
embed_branches = []
df_ints = []
all_inputs = []
all_branch_outputs = []

for col in categorical_headers_ints:
    # encode as ints for the embedding
    #X_ints_train.append( X_train[col].values )
    #X_ints_test.append( X_test[col].values )
    df_ints.append( df[col].values )
    
    # get the number of categories
    N = max(df_ints[-1]+1) # same as the max(df_train[col])
    
    # create embedding branch from the number of categories
    inputs = Input(shape=(1,),dtype='int32')
    all_inputs.append(inputs)
    x = Embedding(input_dim=N, output_dim=int(np.sqrt(N)), input_length=1)(inputs)
    x = Flatten()(x)
    all_branch_outputs.append(x)

# also get a dense branch of the numeric features
all_inputs.append(Input(shape=(df_num.shape[1],),sparse=False))
x = Dense(units=20, activation='relu')(all_inputs[-1])
all_branch_outputs.append( Dense(units=10,activation='relu')(x) )

# merge the branches together
final_branch = concatenate(all_branch_outputs)
final_branch = Dense(units=1,activation='sigmoid')(final_branch)

model = Model(inputs=all_inputs, outputs=final_branch)

model.compile(optimizer='sgd',
              loss='mean_squared_error',
              metrics=['accuracy'])

model.fit(df_ints + [df_num],
        y_train, epochs=10, batch_size=32, verbose=1)


ValueError: Input arrays should have the same number of samples as target arrays. Found 66221 input samples and 76324 target samples.

In [116]:
msk = np.random.rand(len(df)) < 0.8
train = df[msk]
test = df[~msk]
print(len(train))
print(len(test)/len(train))

52737
0.25568386521796843


In [117]:
train, test = train_test_split(df, test_size=0.2)